In [1]:
# Imports

from sklearn import datasets,ensemble,metrics
import pdb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib.colors import ListedColormap
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
################################################### LAB 2+3 ###################################################

In [3]:
def get_MNIST_train():
    
    mnist_train_data = np.zeros([60000,784])
    mnist_train_labels = np.zeros(60000)
    
    f = open('train-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('train-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4bytes magic number, 4 bytes nr labels
    
    mnist_train_data = np.fromfile(f,dtype=np.uint8).reshape(60000,784)
    mnist_train_data = mnist_train_data.reshape(60000,1,28,28)
    mnist_train_labels = np.fromfile(g,dtype=np.uint8)
    
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_train_data = mnist_train_data.astype(np.float32)
    mnist_train_labels = mnist_train_labels.astype(np.int64)
    
    return mnist_train_data, mnist_train_labels

In [4]:
def get_MNIST_test():
    
    mnist_test_data = np.zeros([10000,784])
    mnist_test_labels = np.zeros(10000)
    
    f = open('t10k-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4bytes magic number, 4 bytes nr labels
    
    mnist_test_data = np.fromfile(f,dtype=np.uint8).reshape(10000,784)
    mnist_test_data = mnist_test_data.reshape(10000,1,28,28)
    mnist_test_labels = np.fromfile(g,dtype=np.uint8)
        
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_test_data = mnist_test_data.astype(np.float32)
    mnist_test_labels = mnist_test_labels.astype(np.int64)
    
    return mnist_test_data, mnist_test_labels

In [5]:
# MLP cu PyTorch

class Retea_MLP(nn.Module):
    
    def __init__(self, nr_neuroni_input, nr_neuroni_hidden, nr_clase):
        
        # Pentru a putea folosi mai departe reteaua, este recomandata mostenirea
        # clasei de baza nn.Module
        super(Retea_MLP,self).__init__()
        
        self.hidden_layer = nn.Linear(nr_neuroni_input, nr_neuroni_hidden)
        self.relu = nn.ReLU()
        self.out_layer = nn.Linear(nr_neuroni_hidden, nr_clase)
        
    def forward(self,input_batch):
        # Intr-un MLP, intrarea este sub forma unui vector, deci un batch
        # este o matrice de dimensiunea nr_esantioane_batch x dimensiune esantion
        input_batch = torch.from_numpy(input_batch)
        
        # Flatten the input: (batch_size, 1, 28, 28) -> (batch_size, 784)
        input_batch = input_batch.view(input_batch.shape[0], -1)
        hidden = self.relu(self.hidden_layer(input_batch))
        out = self.out_layer(hidden)
        
        return out

In [6]:
# CNN

class Retea_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Definirea straturilor CNN
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        
        self.relu = nn.ReLU()

        # Definirea stratului fully connected
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128,10)
        
    def forward(self, x):

        # Definirea propagarii forward
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))

        x = x.view(x.size(0), -1)  # Flattening

        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [7]:
# Train MLP pe MNIST
    
# Instantiem reteaua
mlp = Retea_MLP(28*28,1000,10)

# Specificarea functiei loss
loss_function = nn.CrossEntropyLoss(reduction='sum')

# Specificarea optimizatorului
optim = torch.optim.Adam(mlp.parameters(), lr=1e-3)

train_data, train_labels = get_MNIST_train()
batch_size = 128 # Se poate si mai mult in cazul curent, dar este o valoare frecventa
nr_epoci = 15
nr_iteratii = train_data.shape[0] // batch_size # Din simplitate, vom ignora ultimul batch, daca este incomplet


for ep in range(nr_epoci):
    predictii = []
    etichete = []

    for it in range(nr_iteratii):
        # Luam urmatoarele <batch_size> esantioane si etichete
        batch_data = train_data[it*batch_size : it*batch_size+batch_size, :]
        batch_labels = train_labels[it*batch_size : it*batch_size+batch_size]
        # Se calculeaza predictia retelei pentru datele curente (forward pass/ propagare inainte)
        current_predict = mlp.forward(batch_data)

        # Se calculeaza valoarea momentana a functiei loss
        loss = loss_function(current_predict, torch.from_numpy(batch_labels)) 
        
        # Se memoreaza predictiile si etichetele aferente batch-ului actual (pentru calculul acuratetii)
        current_predict = np.argmax(current_predict.detach().numpy(), axis=1)
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,batch_labels))
        
        # Antrenarea propriu-zisa
        
            # 1. Se sterg toti gradientii calculati anteriori, pentru toate variabilele antrenabile
            # deoarece, metoda <backward> acumuleaza noile valori, in loc sa le inlocuiasca.
        optim.zero_grad()
            # 2. Calculul tuturor gradientilor. Backpropagation
        loss.backward()
            # 3. Actualizarea tuturor ponderilor, pe baza gradientilor.
        optim.step()
        
        

    # Calculam acuratetea
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

    # Se genereaza o permutare noua a tuturor esantioanelor si etichetelor corespunzatoare
    perm = np.random.permutation(train_data.shape[0])
    train_data = train_data[perm,:]
    train_labels = train_labels[perm]
    
    
    
test_data, test_labels = get_MNIST_test()
batch_size_test = 100 # pentru usurinta, ca sa testam toate etichetele alegem un divizor al numarului de date de test
nr_iteratii_test = test_data.shape[0] // batch_size_test
    
predictii = []
for it in range(nr_iteratii_test):
    batch_data = test_data[it*batch_size_test : it*batch_size_test+batch_size_test, :]
    batch_labels = test_labels[it*batch_size_test : it*batch_size_test+batch_size_test]

    current_predict = mlp.forward(batch_data)
    current_predict = np.argmax(current_predict.detach().numpy(),axis=1)
    predictii = np.concatenate((predictii,current_predict))

acc = np.sum(predictii==test_labels)/len(predictii)
print( 'Acuratetea la test este {}%'.format(acc*100) )

Acuratetea la epoca 1 este 91.60823985042735%
Acuratetea la epoca 2 este 96.60790598290599%
Acuratetea la epoca 3 este 97.04026442307693%
Acuratetea la epoca 4 este 97.27063301282051%
Acuratetea la epoca 5 este 97.21554487179486%
Acuratetea la epoca 6 este 97.2255608974359%
Acuratetea la epoca 7 este 97.0302483974359%
Acuratetea la epoca 8 este 97.21554487179486%
Acuratetea la epoca 9 este 97.29400373931624%
Acuratetea la epoca 10 este 97.61785523504274%
Acuratetea la epoca 11 este 97.96507745726495%
Acuratetea la epoca 12 este 97.9400373931624%
Acuratetea la epoca 13 este 98.00347222222221%
Acuratetea la epoca 14 este 98.17040598290599%
Acuratetea la epoca 15 este 98.19377670940172%
Acuratetea la test este 96.38%


In [8]:
# Train CNN with MNIST Dataset

class DatasetMNIST(Dataset):
    def __init__(self):

        mnist_train_data,mnist_train_labels = get_MNIST_train()

        self.data = mnist_train_data
        self.labels = mnist_train_labels

    def __getitem__(self, index):
        data_point = self.data[index, ...]
        label_point = self.labels[index, ...]

        return {'data': data_point, 'label': label_point}
    
    def __len__(self):
        return self.data.shape[0]
    
###

mnistdataset = DatasetMNIST()
mnistdataloader = DataLoader(mnistdataset, batch_size=128, shuffle=True, num_workers=0)
loss_function = nn.CrossEntropyLoss(reduction='sum')
simple_CNN = Retea_CNN().to("cuda")
optim = torch.optim.SGD(simple_CNN.parameters(), lr=1e-5)

for ep in range(15):

    predictii = []
    etichete = []

    for batch in mnistdataloader:
        data = batch['data'].to("cuda")
        label = batch['label'].to("cuda")

        # print(data.shape)
        # print(label.shape)

        # data_as_image = data[0, ...].cpu().detach().numpy().squeeze()
        # print(data_as_image.shape)

        # plt.imshow(data_as_image, cmap='gray')
        # continue

        current_predict = simple_CNN.forward(data)

        # Se calculeaza valoarea momentana a functiei loss
        loss = loss_function(current_predict, label) 
        
        # Se memoreaza predictiile si etichetele aferente batch-ului actual (pentru calculul acuratetii)
        current_predict = np.argmax(current_predict.cpu().detach().numpy(), axis=1)
        label = label.cpu().detach().numpy()

        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,label))

        optim.zero_grad()
            # 2. Calculul tuturor gradientilor. Backpropagation
        loss.backward()
            # 3. Actualizarea tuturor ponderilor, pe baza gradientilor.
        optim.step()
        
    # Calculam acuratetea
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

Acuratetea la epoca 1 este 89.91166666666666%
Acuratetea la epoca 2 este 96.47166666666666%
Acuratetea la epoca 3 este 97.47333333333333%
Acuratetea la epoca 4 este 97.93166666666666%
Acuratetea la epoca 5 este 98.25500000000001%
Acuratetea la epoca 6 este 98.46166666666667%
Acuratetea la epoca 7 este 98.67166666666667%
Acuratetea la epoca 8 este 98.81333333333333%
Acuratetea la epoca 9 este 98.91499999999999%
Acuratetea la epoca 10 este 99.02166666666666%
Acuratetea la epoca 11 este 99.11666666666666%
Acuratetea la epoca 12 este 99.19%
Acuratetea la epoca 13 este 99.24666666666667%
Acuratetea la epoca 14 este 99.32666666666667%
Acuratetea la epoca 15 este 99.39833333333333%


In [9]:
#################################################### LAB 4 ####################################################

In [11]:
def get_MNIST_train():
    
    mnist_train_data = np.zeros([60000,784]) # fiecare linie este o img 28x28
    mnist_train_labels = np.zeros(60000)
    
    # ~ url = 'https://github.com/golbin/TensorFlow-MNIST/tree/master/mnist/data/train-images-idx3-ubyte.gz'
    # ~ r = requests.get(url, allow_redirects=True)
    # ~ open('train-images-idx3-ubyte.gz', 'wb').write(r.content)
    
    # ~ url = 'https://github.com/golbin/TensorFlow-MNIST/tree/master/mnist/datatrain-images-idx3-ubyte.gz'
    # ~ r = requests.get(url, allow_redirects=True)
    # ~ open('train-labels.idx1-ubyte.gz', 'wb').write(r.content)
    
    f = open('train-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('train-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_train_data = np.fromfile(f,dtype=np.uint8).reshape(60000,784) # modify here
    mnist_train_labels = np.fromfile(g,dtype=np.uint8)
        
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_train_data = mnist_train_data.astype(np.float32)
    mnist_train_labels = mnist_train_labels.astype(np.int64)
        
    return mnist_train_data, mnist_train_labels

def get_MNIST_test():
    
    mnist_test_data = np.zeros([10000,784])
    mnist_test_labels = np.zeros(10000)
    
    f = open(r'Lab IV\t10k-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open(r'Lab IV\t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_test_data = np.fromfile(f,dtype=np.uint8).reshape(10000,784) # modify here
    mnist_test_labels = np.fromfile(g,dtype=np.uint8)
    
    # Conversii pentru a se potrivi cu procesul de testare    
    mnist_test_data = mnist_test_data.astype(np.float32)
    mnist_test_labels = mnist_test_labels.astype(np.int64)        
    
    return mnist_test_data, mnist_test_labels

In [12]:
# Primele 200 imagini din setul de antrenare MNIST

from torch.utils.data import Dataset, DataLoader
class DatasetMNIST(Dataset):
    def __init__(self):

        mnist_train_data,mnist_train_labels = get_MNIST_train()

        self.data = mnist_train_data[:200]
        self.labels = mnist_train_labels[:200]

    def __getitem__(self, index):
        data_point = torch.from_numpy(self.data[index, ...])
        label_point = torch.from_numpy(self.labels[index, ...])

        return {'data': data_point, 'label': label_point}
    
    def __len__(self):
        return self.data.shape[0]

In [13]:
# 2 straturi ascunse (3 straturi fully connected) + BatchNorm

class ReteaMLP(nn.Module):
    def __init__(self):
        super().__init__()
 
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(28*28, 512)
        # self.drop_x = nn.Dropout(p=0.5)
        self.bn_x = nn.BatchNorm1d(num_features = 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
 
    def forward(self, x):
        x = self.relu(self.fc1(x))
        # x = self.drop_x(x)
        x = self.bn_x(x)
        
        x = x.view(x.size(0), -1)  # Flattening
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
 
        return x

In [15]:
# Train MLP pe MNIST cu BatchNorm, 512 neuroni in primul strat ascuns, 128 in al doilea strat ascuns
# 40 epoci

mnistdataset = DatasetMNIST()
mnist_loader = DataLoader(mnistdataset, batch_size=128, shuffle=True, num_workers=0)
simple_CNN = ReteaMLP()
loss_function = nn.CrossEntropyLoss(reduction='sum')
optim = torch.optim.SGD(simple_CNN.parameters(), lr=1e-4)

for ep in range(40):
 
    predictii = []
    etichete = []
 
    for batch in mnist_loader:
        data, label = batch['data'], batch['label']
 
        current_predict = simple_CNN.forward(data)
 
        loss = loss_function(current_predict, label)
 
        current_predict = np.argmax(current_predict.detach().numpy(), axis=1)
 
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,label))
 
        optim.zero_grad()

        loss.backward()

        optim.step()

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer = optim, step_size = 20, gamma=0.1)
 
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

Acuratetea la epoca 1 este 10.5%
Acuratetea la epoca 2 este 17.5%
Acuratetea la epoca 3 este 27.500000000000004%
Acuratetea la epoca 4 este 44.5%
Acuratetea la epoca 5 este 54.50000000000001%
Acuratetea la epoca 6 este 69.5%
Acuratetea la epoca 7 este 73.0%
Acuratetea la epoca 8 este 76.0%
Acuratetea la epoca 9 este 80.0%
Acuratetea la epoca 10 este 83.0%
Acuratetea la epoca 11 este 85.5%
Acuratetea la epoca 12 este 86.5%
Acuratetea la epoca 13 este 86.5%
Acuratetea la epoca 14 este 89.0%
Acuratetea la epoca 15 este 88.5%
Acuratetea la epoca 16 este 90.0%
Acuratetea la epoca 17 este 90.5%
Acuratetea la epoca 18 este 92.0%
Acuratetea la epoca 19 este 92.0%
Acuratetea la epoca 20 este 93.0%
Acuratetea la epoca 21 este 92.0%
Acuratetea la epoca 22 este 93.0%
Acuratetea la epoca 23 este 94.5%
Acuratetea la epoca 24 este 94.0%
Acuratetea la epoca 25 este 94.5%
Acuratetea la epoca 26 este 94.5%
Acuratetea la epoca 27 este 94.5%
Acuratetea la epoca 28 este 94.0%
Acuratetea la epoca 29 este 95

In [16]:
# transformari pentru AlexNet (augmentare + normalizare)

from torchvision import transforms

class DatasetMNIST(Dataset):
    def __init__(self):

        mnist_train_data,mnist_train_labels = get_MNIST_train()

        self.data = mnist_train_data[:200]
        self.labels = mnist_train_labels[:200]

        self.transf = transforms.Compose([
            transforms.ToPILImage(), transforms.Resize([224,224]),
            transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def __getitem__(self, index):
        data_point = torch.from_numpy(self.data[index, ...])
        label_point = torch.from_numpy(self.labels[index, ...])

        data_point = np.tile(data_point[..., None], (1,1,3))

        data_point = self.transf(data_point)

        label_point = torch.tensor(label_point,dtype=torch.long)

        return {'data': data_point, 'label': label_point}
    
    def __len__(self):
        return self.data.shape[0]

In [17]:
# Model AlexNet pre-antrenat

from torchvision import models

cnn = models.alexnet(pretrained=True)
print(cnn)

C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
for param in cnn.features.parameters():
    param.requires_grad = False # nu se mai actualizeaza ponderile

cnn.classifier[6] = nn.Linear(4096,10) # inlocuim ultimul strat fully-connected

In [ ]:
mnistdataset = DatasetMNIST()
mnist_loader = DataLoader(mnistdataset, batch_size=128, shuffle=True, num_workers=0)
simple_CNN = cnn
loss_function = nn.CrossEntropyLoss(reduction='sum')
optim = torch.optim.SGD(simple_CNN.parameters(), lr=1e-4)

# Bucla de invatare
     
for ep in range(40):
 
    predictii = []
    etichete = []
 
    for batch in mnist_loader:
        data, label = batch['data'], batch['label']
 
        current_predict = simple_CNN.forward(data.float())
 
        loss = loss_function(current_predict, label)
 
        current_predict = np.argmax(current_predict.detach().cpu().numpy(), axis=1)
 
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,label))
 
        optim.zero_grad()
 
        loss.backward()
 
        optim.step()
 
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

TypeError: Input type float32 is not supported